<a href="https://colab.research.google.com/github/michelmalki/MichelMalki/blob/main/CCCS_680_Assignment_3_Question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("CCCS 680 Assignment 3 Question 1 Template") \
       .getOrCreate()

spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
48 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [78]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [79]:
visitors_df = spark.read.csv('/content/drive/My Drive/CCCS-680 Assignment 3/100000_White_House_Visitor_Records_Requests.csv', inferSchema=True, header=False, sep='\t')

In [81]:
visitors_df.show(30)

+--------------------+
|                 _c0|
+--------------------+
|NAMELAST,NAMEFIRS...|
|Adamopoulos,Stell...|
|Brosman,Muriel,,,...|
|Brumfield,Avery,,...|
|Chipman,Catherine...|
|Chubb,Steven,,,,,...|
|Colasante,Lydia,,...|
|Conway,Gavin,,,,,...|
|Cox,Conrad,,,,,,,...|
|Curtis,Shannon,,,...|
|Day,Preston,,,,,,...|
|Dixon,Kelly,,,,,,...|
|duPont,Victor,,,,...|
|Erkenbrecher,Benj...|
|Evans,Nancy,,,,,,...|
|Ferrell,Joshua,,,...|
|Finlay,Jasmine,,,...|
|Green,Joshua,,,,,...|
|Green,Michael,,,,...|
|Hardin,Luke,,,,,,...|
|Homer,Jeffrey,,,,...|
|Homer,Matthew,,,,...|
|Kane,Sara,,,,,,,,...|
|Kellman,Niyah,,,,...|
|Kellman,Nyree,,,,...|
|Kirby,Emma,,,,,,,...|
|Knox,Derrick,,,,,...|
|Larnick,Nina,,,,,...|
|Laurent,Joanne,,,...|
|Lee,Arman,,,,,,,,...|
+--------------------+
only showing top 30 rows



# 2.A
The 5 most frequent visitors (NAMELAST, NAMEFIRST, NAMEMID) to the
White House.


In [82]:
visitors_df = visitors_df.select("_c0")
visitor_counts = visitors_df.groupBy("_c0").count()
top_visitors = visitor_counts.orderBy(visitor_counts["count"].desc()).limit(5)
top_visitors.show()


+--------------------+-----+
|                 _c0|count|
+--------------------+-----+
|BARRAZA,ALEJANDRO...|    3|
|SMITH,WALTER,H,U0...|    3|
|HOLLY,KHUFU,M,U14...|    2|
|ALMADA,GRACIELA,,...|    2|
|CARSON,JOHN,C,OPE...|    2|
+--------------------+-----+



# 2.B
The 5 most frequently visited people (VISITEE_NAMELAST, VISITEE_NAMEFIRST)
in the White House.

In [83]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, count

visitor_counts = visitors_df .groupBy("_c0").agg(count("*").alias("visit_count"))
top_5_visitors = visitor_counts.orderBy("visit_count", ascending=False).limit(5)

print("The 5 most frequent visitors to the White House:")
top_5_visitors.show(truncate=False)


spark.stop()

The 5 most frequent visitors to the White House:
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|_c0                                                                                                                                                                                                                  |visit_count|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|BARRAZA,ALEJANDRO,,U08957,,VA,,,,,05/21/2010 02:44:00 PM,5/28/10 11:00,5/28/10 23:59,,230,B9,WIN,5/21/10 14:44,B9,OFFICE,VISITORS,WH,RESIDENCE,OFFICE,VISITORS,,TOURS ,08/27/2010 07:00:00 AM +0000                  |3          |
|SMITH,WALTER,H,U03679,,VA,,,,,05/13/20